In [2]:
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.preprocessing.image import load_img, array_to_img
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
titanic_orig = pd.read_csv('train.csv')
titanic_test_orig = pd.read_csv('test.csv')

In [4]:
def transform(data):    
    data['hasCabin'] = data['Cabin'].notna()
    data = data.drop(['PassengerId', 'Name', 'Ticket','Cabin'],axis=1)

    data = pd.get_dummies(data)
    data.Age = data.Age.fillna(data.Age.mean())
    data.Fare = data.Fare.fillna(data.Fare.mean())
    return data

In [5]:
titanic = transform(titanic_orig)
titanic_test = transform(titanic_test_orig)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(titanic.drop(['Survived'],axis=1),titanic['Survived'],test_size=0.2,random_state=0)

In [8]:
y_train=to_categorical(y_train,2)
y_test=to_categorical(y_test,2)

In [37]:
type(y_train)

numpy.ndarray

In [36]:
X_train.loc[0:5], y_train.loc(0:5)

SyntaxError: invalid syntax (<ipython-input-36-6f8369d910bd>, line 1)

In [23]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(11,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(2,activation='softmax'))

In [24]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 64)                768       
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 130       
Total params: 5,058
Trainable params: 5,058
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 712 samples, validate on 179 samples
Epoch 1/10
712/712 [==============================] - 0s 138us/step - loss: 1.6093 - accuracy: 0.5604 - val_loss: 0.6980 - val_accuracy: 0.7542
Epoch 2/10
712/712 [==============================] - 0s 37us/step - loss: 0.6878 - accuracy: 0.6433 - val_loss: 0.5939 - val_accuracy: 0.6704
Epoch 3/10
712/712 [==============================] - 0s 37us/step - loss: 0.5799 - accuracy: 0.7022 - val_loss: 0.5467 - val_accuracy: 0.7318
Epoch 4/10
712/712 [==============================] - 0s 39us/step - loss: 0.5908 - accuracy: 0.6910 - val_loss: 0.5646 - val_accuracy: 0.7039
Epoch 5/10
712/712 [==============================] - 0s 37us/step - loss: 0.5396 - accuracy: 0.7444 - val_loss: 0.4891 - val_accuracy: 0.7765
Epoch 6/10
712/712 [==============================] - 0s 36us/step - loss: 0.5504 - accuracy: 0.7275 - val_loss: 0.5430 - val_accuracy: 0.7039
Epoch 7/10
712/712 [==============================] - 0s 36us/step - loss: 0.5322 - accuracy: 0

In [41]:
result = pd.DataFrame(model.predict(titanic_test), columns=['one','two'])

In [45]:
result['Survived'] = (result['one'] < result['two']).map({True: 1, False: 0})

In [46]:
result['Survived']

0      0
1      0
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64

In [47]:
# PassengerId,
submission = pd.DataFrame(titanic_test_orig['PassengerId'])
submission.info()
submission['Survived'] = result['Survived']
submission.to_csv('gender_submission.csv',index=False)
submission.to_csv('/Users/a0a00uj/Downloads/gender_submission.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
dtypes: int64(1)
memory usage: 3.4 KB


In [48]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
